## Imports

In [1]:
import os
import sys
import json
import gc
import re
import json
from tqdm import notebook as tqdm
import pickle
import itertools
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import httpx
import logging

from llm_input import top_tokens_lvl0, top_tokens_lvl1, gpt_output_lvl0, gpt_output_lvl1

OLLAMA_CONFIG = {
    "keep_alive": "5m",
    "stream": False,
}

SEP_TOKEN = '<sep>'

gpt_output_lvl0 = np.array(list(gpt_output_lvl0.items()))
gpt_output_lvl1 = np.array(list(gpt_output_lvl1.items()))

In [2]:
sns.set(font_scale=1.2, palette='Set2')

plt.rcParams['font.family'] = 'DejaVu Serif'
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.markersize'] = 12
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 24
plt.rcParams['legend.fontsize'] = 24
plt.rcParams['axes.titlesize'] = 30
plt.rcParams['axes.labelsize'] = 24
plt.rcParams["figure.figsize"] = (12, 7)

SEED = 123

## Text preprocessing

Смотрим на распределение секций внутри текста.

Секция -- всё, что обёрнуто в `==`

Из примеров выше: `== Майндмап ==`, `== Аннотация ==`, `== Видео ==`

Далее фильтруем по числу документов в секции: их должно быть не менее 20.

В конце сортируем по числу документов в секции.

In [3]:
with open('../data/in/0x1tv-dataset.pickle', 'rb') as fd:
    data = pickle.load(fd)

Оставляем 3 домена: заголовок, аннотация и тезисы

In [4]:
def get_min_position(*positions):
    return min([pos for pos in positions if pos >= 0], default=len(data))

def extract_section_text(data, section_name):
    if section_name == 'Аннотация':
        blockquote_pattern = r'<blockquote>(.*?)</blockquote>'
        match = re.search(blockquote_pattern, data, re.DOTALL)
        return match.group(1).strip() if match else None
    
    section_pattern = fr'==\s*{re.escape(section_name)}\s*=='
    section_start = re.search(section_pattern, data)
    
    if not section_start:
        return None
   
    position = section_start.end()

    # Look for stop tokens and the start of next sections
    stop_tokens = ['{{----}}', '{{LinksSection}}', '== Примечания и отзывы ==']
    ends = [data.find(token, position) for token in stop_tokens]
    end_position = get_min_position(*ends)
    
    if end_position != len(data):
        return data[position:end_position].strip()
    
    return data[position:].strip()

def extract_sections(data):
    sections = ['Аннотация', 'Thesis', 'Тезисы', 'Расширенные тезисы']
    extracted_texts = {section: extract_section_text(data, section) for section in sections}
    return extracted_texts


def coalesce(dct, keys):
    for key in keys:
        if dct[key] is not None or len(dct[key]) > 0:
            return dct[key]
    return None


def parse_document(doc):
    parsed_doc = dict()
    parsed_doc['title'] = doc['title']
    
    sections_data = extract_sections(doc['text'])
    parsed_doc['annotation'] = sections_data['Аннотация']
    parsed_doc['thesis'] = coalesce(sections_data, ['Thesis', 'Тезисы', 'Расширенные тезисы'])
    
    return parsed_doc

Парсим ФИО докладчиков

In [20]:
def speaker_detect(text):
    speakers = re.findall(r'(?<={{Speaker\|).*(?=}})', text)
    preproc_speakers = []
    for speaker in speakers:
        preproc_speakers.extend(speaker.split('|'))
    return preproc_speakers


all_speakers = [speaker_detect(text['text']) for text in data['articles']]

print('Число докладов без спикеров:', len(list(filter(lambda x: len(x) == 0, all_speakers))))
print('Число докладов с одним спикером:', len(list(filter(lambda x: len(x) == 1, all_speakers))))
print('Число докладов с двумя спикерами:', len(list(filter(lambda x: len(x) == 2, all_speakers))))
print('Число докладов с не менее тремя спикерами:', len(list(filter(lambda x: len(x) >= 3, all_speakers))))

Число докладов без спикеров: 74
Число докладов с одним спикером: 2313
Число докладов с двумя спикерами: 119
Число докладов с не менее тремя спикерами: 35


Фильтруем категории

In [21]:
def clear_categories(sample):
    return [re.sub('Категория:', '', el) for el in sample]


text_categories = [clear_categories(item['categories']) for item in data['articles']]
child_categories = [item['title'] for item in data['categories']]
parent_categories = [clear_categories(item['categories']) for item in data['categories']]

ALL_CATEGORIES = list(set(itertools.chain(*text_categories)) | set(child_categories) | set(itertools.chain(*parent_categories)))

child_to_parent_categories = {item['title']: clear_categories(item['categories']) for item in data['categories']}

In [23]:
ALL_CATEGORIES

['Геймификация в UX',
 'SECON-2017',
 'Михаил Карпов',
 'Никита Ермаков',
 'Илья Богунов',
 'Игорь Буренков',
 'Александр Прозоров',
 'Дмитрий Сатин',
 'Олег Громов',
 'Виталий Камянский',
 'Евгений Кабанов',
 'Юрий Орлов',
 'Наталия Смирнова',
 'Владимир Вахлов',
 'Михаил Заборов',
 'Наим Шафиев',
 'UI бизнес-приложений',
 'Оценка сотрудников',
 'Виктория Придатко',
 'Zuzi Sochova',
 'Константин Рыбас',
 'Екатерина Потапова',
 'Дмитрий Подлужный',
 'Федор Ляхов',
 'Алексей Лесовский',
 'Кристина Пивоварова',
 'Платон Днепровский',
 'Голосовой интерфейс',
 'Никита Фролов',
 'Александр Легалов',
 'Анастасия Маркина',
 'Дмитрий Петерсон',
 'Сергей Егоров',
 'Владислав Рябченко',
 'Денис Доронин',
 'User Story',
 'Михаил Трутнев',
 'Мотивация',
 'САПР',
 'Дмитрий Мартынов',
 'Денис Петров',
 'Артур Арсенов',
 'Стас Павлов',
 'OSEDUCONF-2019',
 'Linux-дистрибутивы для Enterprise',
 'Waterfall',
 'Елена Сагалаева',
 'Сергей Гельметдинов',
 'Наталья Зиновьева',
 'Инга Егорова',
 'Илья Каштан

Удалим вручную все имена из списка категорий + `HasSpeaker`

🧔: Ох, вручную ничего делать нельзя, невоспроизводимо и немодифицируемо. Да и вручную не выйдет («Pascale Xelot-Dugat» — это спикер)

In [22]:
ALL_CATEGORIES_WITHOUT_NAMES = set([
    '.NET',
    '1C',
    'ALT Linux',
    'ALTLinux на Эльбрусе',
    'AR',
    'AWS',
    'Accessibility',
    'Agile',
    'Agile Introduction',
    'Agile process',
    'Agile в корпорациях',
    'Agile — технологические практики',
    'Agile&Lean Mindset',
    'Agile-культура',
    'Agile-масштабирование',
    'Agile-преобразования',
    'Alfresco',
    'Ansible',
    'Arduino',
    'AstraLinux',
    'Atlassian',
    'Azure',
    'B2B продукты',
    'BDD',
    'BigData',
    'Blockchain',
    'Bluemix',
    'Business rules engine',
    'C++',
    'CMMI',
    'CQRS',
    'CRIU',
    'Clouds',
    'Clsync',
    'Code Review',
    'Collaboration tools',
    'Configuration Management',
    'ContactOK',
    'Continuous Integration',
    'CouchDB',
    'Csharp',
    'CustisWikiToLib',
    'Customer Journey Map',
    'DDD',
    'DSL-языки',
    'Data Analysis',
    'Deployment',
    'Design Thinking',
    'DevOps',
    'Draft',
    'E-commerce',
    'Embox',
    'Erlang',
    'Extreme Programming',
    'Feature Branches',
    'Firefox',
    'Foresight management',
    'FreeIPA',
    'Front end development',
    'Fsharp',
    'GWT',
    'Glibc',
    'Go',
    'Groovy',
    'Growth Hacking',
    'HR',
    'Hardware',
    'Health',
    'High Performace Computing',
    'Highload-архитектуры',
    'IP-телефония',
    'IT-законы',
    'IT-образование',
    'Impact Map',
    'Information Security',
    'Internet of Thing',
    'Java',
    'Java EE',
    'Javascript',
    'Jenkins',
    'Kanban',
    'Knowledge Management',
    'Kotlin',
    'Kubernetes',
    'LAMP',
    'LSM',
    'LeSS',
    'Lean',
    'Lean Startup',
    'Legacy',
    'Libreoffice',
    'Linux',
    'Linux для Эльбруса',
    'Linux-дистрибутивы',
    'Linux-дистрибутивы для Enterprise',
    'Linux-обучение',
    'Lua',
    'MIPS',
    'Machine Learning',
    'Mechanics',
    'Microsoft',
    'Misc',
    'Mkimage-profiles',
    'MongoDB',
    'Morpheus',
    'MySQL',
    'NLP',
    'NOSQL',
    'NUI',
    'Natural Language Processing',
    'NeedContacts',
    'Nemerle',
    'Node.js',
    'Object Oriented Programming',
    'Open-source',
    'Open-source CAD',
    'Open-source CMS',
    'Open-source CRM',
    'Open-source ERP',
    'Open-source PAAS',
    'Open-source TCO',
    'Open-source and Community',
    'Open-source and hardware',
    'Open-source communications',
    'Open-source for Enterprise',
    'Open-source operating systems',
    'Open-source projects',
    'Open-source СУБД',
    'Open-source и законы',
    'OpenShift',
    'OpenStack',
    'OpenVZ',
    'PAAS',
    'PHP',
    'Pascale Xelot-Dugat',
    'People Management',
    'Pivot',
    'PostgreSQL',
    'ProductMeetup',
    'Python',
    'RISC-V',
    'ROSA Linux',
    'ROSALab',
    'RTOS',
    'Reviewed',
    'Riak',
    'Ruby',
    'RunaWFE',
    'SAP',
    'SELinux',
    'SOLID',
    'SVM',
    'Samba',
    'Scala',
    'Scrum',
    'Serverless',
    'Sharepoint',
    'SkillsWiki',
    'Skype',
    'Software Defined Networks',
    'Strace',
    'Support',
    'TAU-платформа',
    'TDD',
    'Talks in English',
    'Tarantool',
    'Taucraft',
    'Team Communication',
    'Teмы',
    'ToPublish',
    'Tuukka Ahoniemi',
    'UI',
    'UI SmartTV',
    'UI бизнес-приложений',
    'UX',
    'UX + Agile',
    'UX проектирование',
    'User Story',
    'VCS',
    'Visual Studio',
    'Waterfall',
    'WebRTC',
    'Windows',
    'World Usability Day',
    'ZFS',
    'Zabbix',
    'Автоматизированное тестирование',
    'Алгоритмы',
    'Анализ программ и систем',
    'Аналитика',
    'Архитектура',
    'Архитектура информационных систем',
    'Архитектура серверных приложений',
    'Аутентификация и авторизация',
    'БПЛА',
    'Базы данных',
    'Байкал',
    'Безопасность',
    'Бизнес в IT',
    'Бизнес и СПО',
    'Бизнес-анализ',
    'Блиц-доклады',
    'Веб-дизайн',
    'Веб-разработка',
    'Вебинары PingWin',
    'Верификация',
    'Видеосвязь',
    'Визуализация',
    'Виртуализация',
    'Виртуальный ассистент',
    'Встраиваемые системы',
    'Выход на зарубежные рынки',
    'Геймификация',
    'Геймификация в UX',
    'Геолокация',
    'Голосовой интерфейс',
    'Госсектор',
    'Государство и софт',
    'Графовые базы данных',
    'Диаграммы Кано',
    'Дизайн',
    'Динамический анализ',
    'Дискуссии',
    'Дискуссии о юзабилити',
    'Доверенная загрузка',
    'Доклад со стенограммой',
    'Докладчики',
    'Доклады на английском',
    'Доклады на белорусском языке',
    'Доклады на иностранных языках',
    'Доклады на украинском',
    'Документация и Agile',
    'Документирование',
    'Запуск продукта',
    'Запуск продукта в Retail',
    'Инструменты майнтейнера',
    'Инструменты майнтейнеров',
    'Инструменты разработки',
    'Информационная безопасность',
    'Информационная безопасность и СПО',
    'Информационные системы ВУЗов',
    'Использование open-source',
    'Исследовательское тестирование',
    'История из практики',
    'Картография',
    'Карьера в IT',
    'Командообразование',
    'Компиляторы',
    'Компиляция под Linux',
    'Компьютерная графика',
    'Компьютерное зрение',
    'Конференции',
    'Корпоративные решения',
    'Криптография',
    'Кроссплатформенная разработка',
    'Круглый стол',
    'Кумир',
    'Лидерство',
    'Линейки конференций',
    'Локализация',
    'Маркетинг',
    'Мастер-классы',
    'Менеджмент',
    'Метрики качества',
    'Микропрограммирование',
    'Микросервисы',
    'Мобильная разработка',
    'Моделирование бизнес-процессов',
    'Моделирование физических систем',
    'Монетизация',
    'Мониторинг',
    'Мотивация',
    'Наука',
    'Облачные сервисы',
    'Образование',
    'Обучение',
    'Обучение бизнес-анализу',
    'Обучение бизнес-процессам',
    'Обучение проектному менеджменту',
    'Обучение системному программированию',
    'Онлайн-обучение',
    'Операционные системы',
    'Оптимизация приложения',
    'Опыт внедрения СПО',
    'Организационные изменения',
    'Организационный анализ',
    'Открытые данные',
    'Отладка',
    'Оценка сотрудников',
    'Очереди',
    'Параллельное программирование',
    'Пиктомир',
    'Планирование',
    'Планирование в Agile',
    'Планировка задач',
    'Привественные речи',
    'Программирование',
    'Программная архитектура',
    'Продуктовая аналитика',
    'Прототипирование UI',
    'Процесс разработки',
    'Процесс разработки UX и UI',
    'Процесс тестирования',
    'Психология пользователя',
    'Психология разработки',
    'Разработка open-source',
    'Разработка десктоп-приложений',
    'Разработка десктопных приложений под Windows',
    'Разработка игр',
    'Разработка операционных систем',
    'Распределенные системы',
    'Редкие языки программирования',
    'Реклама',
    'Реклама компании',
    'Рекомендательные системы',
    'Ретроспектива',
    'Рефакторинг',
    'Робототехника',
    'С++',
    'САПР',
    'СПО в Госуправлении',
    'СПО в России',
    'СПО в науке',
    'СПО в образовании',
    'СПО в творчестве',
    'СПО для системного администрирования',
    'СУБД',
    'Свободные библиотеки построения графиков',
    'Свободные лицензии',
    'Сервис-ориентированная архитектура',
    'Сети',
    'Системное администрирование',
    'Системное мышление',
    'Системный анализ',
    'Системный подход',
    'Системы управления версиями',
    'Скрытые категории',
    'Собрания ALT.NET',
    'Совещания',
    'Стажировка',
    'Статический анализ кода',
    'Стратегическое планирование',
    'ТРИЗ',
    'Темы',
    'Теория ограничений',
    'Тестирование',
    'Тестирование UI',
    'Тестирование игр',
    'Тестирование мобильных приложений',
    'Тестирование производительности',
    'Технологии',
    'Технологии будущего',
    'Технологии крупных вендоров',
    'Тренды Open-source',
    'Умные вещи',
    'Управление заинтересованными сторонами',
    'Управление качеством',
    'Управление продуктами',
    'Управление рисками',
    'Управление собой',
    'Управление техподдержкой',
    'Управление требованиями',
    'Файловые системы',
    'Философия программирования',
    'Философия юзабилити',
    'Финансовые системы',
    'Фреймворки',
    'Фриланс',
    'Функциональное программирование',
    'Хранение данных',
    'Хэши',
    'Эксплуатация',
    'Эльбрус',
    'Юзабилити',
    'Юзабилити в 1C',
    'Юзабилити в играх',
    'Юзабилити интернет-магазинов',
    'Юзабилити исследования',
    'Юзабилити мобильных устройств',
    'Юзабилити образование',
    'Юзабилити поиска',
    'Юзабилити текста',
    'Языки программирования',
])

In [8]:
child_to_parent_categories

{'.NET': ['Программирование'],
 '1C': [],
 'ADD-2010': ['Application Developers Days', 'Конференции'],
 'ADD-2011': ['Application Developers Days', 'Конференции'],
 'ADD-2012': ['Application Developers Days', 'Конференции'],
 'ALTLinux на Эльбрусе': ['ALT Linux', 'Linux для Эльбруса'],
 'ALT Linux': ['Linux-дистрибутивы для Enterprise'],
 'AR': ['Программирование'],
 'AWS': ['PAAS', 'Облачные сервисы'],
 'Accessibility': ['Юзабилити'],
 'Adrian Reed': ['Докладчики'],
 'Agile': ['Менеджмент'],
 'Agile&Lean Mindset': ['Agile-культура'],
 'Agile-культура': ['Agile'],
 'Agile-масштабирование': ['Agile'],
 'Agile-преобразования': [],
 'AgileDays': ['AgileDays-2011', 'AgileDays-2014', 'Линейки конференций'],
 'AgileDays-2011': ['AgileDays', 'Конференции'],
 'AgileDays-2013': ['AgileDays', 'Конференции'],
 'AgileDays-2014': ['AgileDays', 'Конференции'],
 'AgileDays-2015': ['AgileDays', 'Конференции'],
 'AgileKitchen (2013-10-18)': ['AgileDays', 'Конференции'],
 'Agile Introduction': ['Agile']

In [9]:
def coalesce(dct, keys):
    for key in keys:
        if dct[key] is not None:
            return dct[key]
    return None


def parse_document_with_categories(doc):
    parsed_doc = dict()
    parsed_doc["title"] = doc["title"]
    parsed_doc['speakers'] = speaker_detect(doc['text'])
    
    sections_data = extract_sections(doc['text'])
    parsed_doc["annotation"] = sections_data["Аннотация"]
    parsed_doc["thesis"] = coalesce(
        sections_data, ["Thesis", "Тезисы", "Расширенные тезисы"]
    )
    
    parsed_doc['raw_categories'] = clear_categories(doc["categories"])
    parsed_doc["categories"] = list(
        filter(
            lambda x: x in ALL_CATEGORIES_WITHOUT_NAMES,
            parsed_doc['raw_categories'],
        )
    )
    return parsed_doc


data_raw = [parse_document_with_categories(doc) for doc in data['articles']]

Так как большие языковые модели довольно "умные", то можно спокойно опустить большую часть фильтрации текста.

Причём, если эту фильтрацию оставить, то полученные эмбеддинги окажутся не такими хорошими, поскольку обработанный текст менее логичен и связен, в отличии от естественного языка.

Тем не менее, мы оставим некоторые фильтры: это удаление ссылок и сохранение токена <sep> между разными модальностями

Очистка текста:
- удаление ссылок
- удаление всех HTML тегов

In [10]:
def clean_text_for_llm(text):
    if text is None or len(text) == 0:
        return ""

    text = re.sub(r"<[^>]+>", "", text)
    text = re.sub(r"http\S+|www\.\S+", "", text)

    # Remove LaTeX commands and symbols
    text = re.sub(r'\\[a-zA-Z]+\s*({.*?})?', '', text)
    text = re.sub(r'\{.*?\}', '', text)

    # Remove special characters and excessive whitespace
    text = re.sub(r'[^a-zA-Zа-яА-Я0-9\s.,<>:;\'\"()\[\]\-—]', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    return text


def clean_and_merge_document_for_llm(doc, sep_token=SEP_TOKEN):
    cleaned_texts = []

    # Clean each field and collect non-empty results
    for field in ["title", "annotation", "thesis"]:
        field_text = doc.get(field, "")
        cleaned_tokens = clean_text_for_llm(field_text)

        if cleaned_tokens:
            cleaned_texts.append(cleaned_tokens)

    # Join with <sep> token
    result_text = f" {sep_token} ".join(cleaned_texts).strip()

    result = {
        "text": result_text,
        "categories": doc.get("categories", []),
        "raw_categories": doc.get("raw_categories", []),
        "speakers": doc.get("speakers", []),
    }

    return result


# Example usage
text = """
<html>Пример текста. Это тестовый текст с примером <a href="http://example.com">ссылки</a>.
Вот еще ссылка: https://www.test.ru.
Здесь есть часто-встречающиеся слова, например, "тест" много раз.
...
Также нужно удалить .эти строки.
"""

print(clean_text_for_llm(text))
print(clean_and_merge_document_for_llm(data_raw[1298]))

 Пример текста. Это тестовый текст с примером ссылки. Вот еще ссылка: Здесь есть часто-встречающиеся слова, например, "тест" много раз. ... Также нужно удалить .эти строки. 
{'text': 'Маркетинг мобильных приложений (Юрий Мельничек, SECR-2015) <sep> Я руковожу MAPS.ME с самого начала и до 25 миллионов инсталляций на текущий момент. За это время у меня сформировался системный взгляд на маркетинг мобильных приложений, которым я и поделюсь со слушателями: Какие существуют маркетинговые каналы для раскрутки мобильных приложений (PR, ASO, реклама, фичеринги и т.д). Как работать с каждым из них. Какой путь проходят пользователи до инсталляции приложения. Как увеличить конверсию. Какие есть инструменты мобильного маркетинга. Какое положение маркетинга в бизнес-стратегии мобильного приложения. Доклад будет интересен маркетологам, топ-менеджерам и разработчикам мобильных приложений.', 'categories': ['Мобильная разработка', 'Управление продуктами'], 'raw_categories': ['HasSpeaker', 'SECR-2015', '

Process text

In [11]:
processed_data = [clean_and_merge_document_for_llm(doc) for doc in data_raw]

In [12]:
text_lengths = list(map(len, [elem['text'] for elem in processed_data]))
pd.Series(text_lengths).describe()

count     2541.000000
mean      1327.888233
std       2120.183248
min          2.000000
25%        282.000000
50%        623.000000
75%       1153.000000
max      20089.000000
dtype: float64

## Prompt engineering for topic summary

### Utils

In [13]:
OLLAMA_CONFIG = {
    "keep_alive": "2m",
    "stream": False,
}

SYSTEM_PROMPT = (
    "You are an AI assistant tasked with generating precise and concise topic titles. "
    "Do not include explanations or reasoning—only provide the final title for each topic. "
    "Write answer in Russian language, but if there are foreign terms leave them as is."
)

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [14]:
def run_prompt(endpoint, prompt, model='gemma2', temperature=0, seed=123, **kwargs):
    options_config = {"temperature": temperature, "seed": seed, **kwargs}

    combined_prompt = f"{SYSTEM_PROMPT}\n\n{prompt}"

    payload = {
        **OLLAMA_CONFIG,
        "model": model,
        "prompt": combined_prompt,
        "options": options_config
    }

    response = httpx.post(
        endpoint,
        json=payload,
        headers={"Content-Type": "application/json"},
        timeout=300,
    )
    
    if response.status_code != 200:
        print("Error", response.status_code)
        assert(False)

    return response


def debug_prompt(prompt, endpoint='http://127.0.0.1:11434/api/generate', model='gemma2', **kwargs):
    text = re.compile(r'#[^\n]*').sub('', prompt).strip()
    response = run_prompt(endpoint, text, model, **kwargs)
    response_content = response.json().get('response', 'No response found.')
    
    print(response_content)

### Examples

In [15]:
!ollama list

NAME                               ID              SIZE      MODIFIED     
rscr/ruadapt_qwen2.5_32b:Q8_0      9907b3df7370    34 GB     7 days ago      
mistral:latest                     f974a74358d6    4.1 GB    2 weeks ago     
llama3:latest                      365c0bd3c000    4.7 GB    2 weeks ago     
aya-expanse:latest                 65f986688a01    5.1 GB    7 weeks ago     
gemma2:latest                      ff02c3702f32    5.4 GB    7 weeks ago     
gemma2:27b                         53261bc9c192    15 GB     2 months ago    
mistral-large:latest               0ca7dfa0bf06    69 GB     2 months ago    
deepseek-v2.5:latest               409b2dd8a3c4    132 GB    2 months ago    
dolphin-mixtral:latest             cfada4ba31c7    26 GB     8 months ago    
codellama:latest                   8fdf8f752f6e    3.8 GB    8 months ago    
llama2:latest                      78e26419b446    3.8 GB    8 months ago    
mistral:7b-instruct-v0.2-q4_K_S    ba00d3a5239e    4.1 GB    9 mont

Большие модели (больше 15 GB) долго инференсятся: это `mistral-large`, `deepseek-v2.5`, `dolphin-mixtral`. Поэтому их мы отбросим

Также из рассмотрения исключены `codellama` из-за ошибки 404 в API

In [24]:
models_list = (
    'llama3',
    'mistral',
    'aya-expanse',
    'gemma2',
    'gemma2:27b',
    'llama2',
    'mistral:7b-instruct-v0.2-q4_K_S',
    'codellama:13b'
)

for model in models_list:
    print(model)
    debug_prompt("""
    Ты редактор википедии. 
    Прочти текст ''' 
    «Ковидный период» и недавно наступившая «эра перемен» сильно подстегнула ВУЗовское дистанционное образование. Раньше нормой были «оффлайн» занятия, если повезло, поддерживаемые книжкой лектора, видеозаписями лекций-семинаров, а «дистанционка» была скорее редко терпимым исключением. Сейчас же нормой становится ситуация, когда преподаватель и даже студенты разбросаны по всему миру. Но мало, созвонившись в каком-нибудь zoom-е, начать читать лекцию, и, заглядывая в чёрные квадратики отключённых видеокамер, пытаться понять, слышат ли преподавателя студенты и вообще, что они делают, мучительно надеясь вовлечь их в процесс. Увы, это просто не работает. Ещё хуже с домашней работой — мало давать студентам «книгу» и надеятся, что они её прочтут — всё в этом устаревшем формате (объём, статичность, неудобство «листов», неинтерактивность, неверифицируемость) неэффективно по современным меркам.

    Да, есть масса онлайн сервисов, которые уже долгое время помогали айтишникам и студентам во время созвонов, и которые стали ещё популярнее в «ковидную эру» — разделяемые онлайн доски и документы, гугл-доки и гугл-колаб-ноутбуки, … но после «войны взаимных интернет-блокировок и санкций» на них нельзя расчитывать. Например, в РФ программа «Google Suite for Education» закрылась из-за анти-РФ санкций, с другой стороны, десятки и сотни сервисов перестали работать только из-за того, что использовали кеширование и балансировку через сервис Сloudflare (запрещается РКН). Ситуация продолжает ухудшаться, перспективы неутешительны.

    Только системы с открытым кодом, поднятые на своей инфраструктуре, предсказуемы и защищены от этих «блокад» — блокировок и санкций. Мы расскажем именно о таких системах, успешно применяемых и развиваемых автором при преподавании математических и алгоритмических курсов в ИСПРАН и МФТИ и ведении исследовательской и научной работы в отделе математических методов и алгоритмов ИСПРАН.

    Системы для организации созвонов, «интерактивные доски», системы для одновременного совместного редактирования кода и текстов, «коллаборативные блокноты»… организуют и «классную» и «внеклассную» работы, граница между которыми уже размыта по всему, что как-то связано с математикой и алгоритмами. А их использование позволяет делать новые типы образовательного контента — на замену унылым толстым книжкам и уродливым слайдам приходят «живые лаборатории» с компактными интерактивными материалами, с которыми можно и «поиграть», и привлечь студентов к совместному творчеству нового.
    '''
    Выведи категорию, к которой эта статья относится.
    """, model=model)
    print('='*80)

llama3


2024-12-30 05:25:39,271 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


«Образование в эру цифровых технологий»
mistral


2024-12-30 05:25:41,563 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


 Категория: "Онлайн-образование" или "Использование технологий в образовании"
aya-expanse


...


Категория: Образование
mistral:7b-instruct-v0.2-q4_K_S


2024-12-30 05:26:08,536 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


 Эта статья относится к категории "Онлайн-обучение" или "Интерактивные системы для обучения".
codellama:13b


2024-12-30 05:26:12,206 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"



Компьютерные науки


Лучше себя показали `llama3`, `mistral`, `gemma2:27b`, `mistral:7b-instruct-v0.2-q4_K_S`

У `codellama:13b` совсем дурной выход, поэтому не будем её дальше рассматривать

### Модифицированный промпт

In [ ]:
SUMMARIZING_PROMPT = """
    You are an AI language model designed to analyze topics defined by keywords, bigrams, and categories, and then summarize each topic into a concise, descriptive name. Use the provided information (keywords, bigrams, and categories) to infer the central theme of each topic and generate a short, meaningful title. Follow these guidelines:

    1. Focus on the most relevant and representative keywords and bigrams to identify the primary subject of the topic.
    2. Use the categories to refine and contextualize the theme.
    3. Avoid overly generic titles; aim for specificity and clarity.
    4. Ensure the title reflects the core idea that unites the keywords, bigrams, and categories.

    Here is the input format:
    ```
    Topic: [topic_name]
    Words: [list of words]
    Bigrams: [list of bigrams]
    Categories: [list of categories]
    ```

    And here is an example of the desired output:
    ```
    Input:
    Topic: topic_15
    Words: ['архитектура', 'код', 'платформа', 'сервис', 'технология']
    Bigrams: ['хранилище_данные', 'облачный_сервис', 'распределённый_система', 'модуль_ядро', 'обработка_больший']
    Categories: ['программирование', 'программная_архитектура', 'распределенные_системы', 'виртуализация', 'paas']

    Output:
    "topic_15": "Архитектура программ и облачные сервисы"

    Input:
    Topic: topic_16
    Words: ['модель', 'текст', 'анализ', 'метод', 'алгоритм']
    Bigrams: ['ред_база', 'учебный_материал', 'субд_ред', 'вычислительный_комплекс', 'анализ_текст']
    Categories: ['алгоритмы', 'наука', 'machine_learning', 'natural_language_processing', 'data_analysis']

    Output:
    "topic_16": "Модели и алгоритмы анализа текста"

    Input:
    Topic: topic_17
    Words: ['проблема', 'требование', 'оценка', 'качество', 'заказчик']
    Bigrams: ['управление_требование', 'больший_количество', 'совместный_работа', 'оценка_качество', 'реализация_проект']
    Categories: ['бизнес_анализ', 'процесс_тестирования', 'управление_требованиями', 'дизайн', 'team_communication']

    Output:
    "topic_17": "Управление требованиями и дизайн"

    Input:
    Topic: topic_18
    Words: ['являться', 'любой', 'образ', 'позволять', 'часть']
    Bigrams: ['возможность_использование', 'высокий_уровень', 'данный_подход', 'код_проект', 'адресный_пространство']
    Categories: ['программирование', 'java', 'разработка_операционных_систем', 'моделирование_бизнес_процессов', 'доклад_со_стенограммой']

    Output:
    "topic_18": "Подходы в программировании и моделировании"

    Input:
    Topic: topic_19
    Words: ['компания', 'технический', 'российский', 'развитие', 'россия']
    Bigrams: ['команда_разработчик', 'ит_компания', 'технический_писатель', 'свой_проект', 'технический_долг']
    Categories: ['agile_process', 'государство_и_софт', 'документирование', 'спо_в_россии', 'планирование']

    Output:
    "topic_19": "Технические проекты в России"
    ```

    Now, process the following topics and generate titles for each:
"""

In [ ]:
top_tokens_lvl0_arr = list(map(lambda x: x.strip(), top_tokens_lvl0.split('='*80)))
top_tokens_lvl1_arr = list(map(lambda x: x.strip(), top_tokens_lvl1.split('='*80)))

top_tokens_lvl0_arr = np.array(list(filter(None, top_tokens_lvl0_arr)))
top_tokens_lvl1_arr = np.array(list(filter(None, top_tokens_lvl1_arr)))

np.random.seed(123)
lvl0_rand_ids = np.random.choice(len(top_tokens_lvl0_arr), 2, replace=False)
lvl1_rand_ids = np.random.choice(len(top_tokens_lvl1_arr), 2, replace=False)

rand_samples = list(
    itertools.chain(
        *[
            top_tokens_lvl0_arr[lvl0_rand_ids],
            top_tokens_lvl1_arr[lvl1_rand_ids]
        ]
    )
)
rand_samples_gpt_target = list(
    itertools.chain(
        *[
            gpt_output_lvl0[lvl0_rand_ids],
            gpt_output_lvl1[lvl1_rand_ids]
        ]
    )
)

In [ ]:
models_list_v2 = (
    'llama3',
    'mistral',
    'aya-expanse',
    'gemma2',
    'gemma2:27b',
    'llama2',
    'mistral:7b-instruct-v0.2-q4_K_S',
)

print(*rand_samples, sep='\n')
print('\n')
print(*rand_samples_gpt_target, sep='\n')
print('\n\n')

for model in models_list_v2:
    print(model)
    
    for rand_sample in rand_samples:
        debug_prompt(SUMMARIZING_PROMPT + f"```\n{rand_sample}\n```", model=model)
    
    print('='*80)

Topic: topic_14
    Words: ['системный', 'ядро', 'вызов', 'версия', 'сборка']
    Bigrams: ['начинать_версия', 'версия_выпустить', 'fault_injection', 'накладный_расход', 'версия_ядро']
    Categories: ['системное_администрирование', 'devops', 'agile__технологические_практики', 'анализ_программ_и_систем', 'статический_анализ_кода']
Topic: topic_5
    Words: ['пакет', 'дистрибутив', 'объект', 'состояние', 'рамка']
    Bigrams: ['использование_технология', 'рамка_доклад', 'подход_решение', 'сложный_задача', 'проблема_являться']
    Categories: ['параллельное_программирование', 'linux_дистрибутивы', 'alt_linux', 'linux_дистрибутивы_для_enterprise', 'ui']
Topic: topic_4
    Words: ['данные', 'бд', 'база', 'обработка', 'файл']
    Bigrams: ['база_данные', 'обработка_данные', 'принимать_решение', 'головной_боль', 'приложение_позволять']
    Categories: ['конференции', 'front_end_development', 'собрания_altnet']
Topic: topic_90
    Words: ['код', 'должный', 'реализация', 'полный', 'важно']
   

2024-12-20 13:52:57,734 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


"topic_14": "Системное администрирование и версии ядра"


2024-12-20 13:52:58,421 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


"topic_5": "Пакеты и дистрибутивы Linux"


...

 "topic_5": "Пакеты и рамы для дистрибутивов Linux"

```russian
Topic: topic_6
    Words: ['пользовательский', 'интерфейс', 'оптимизация', 'производительность', 'приложение']
    Bigrams: ['создание_приложения', 'высокая_производительность', 'оптимизация_пользовательский_интерфейс', 'работа_приложением', 'интерактивный_интерфейс']
    Categories: ['ui_design', 'web_development', 'mobile_app_development', 'программное_обеспечение', 'performance']

Output:
"topic_6": "Разработка приложений с оптимизированным пользовательским интерфейсом"
```

```russian
Topic: topic_7
    Words: ['процесс', 'технология', 'инновация', 'развитие', 'система']
    Bigrams: ['инновационный_процесс', 'технология_разработки_предметно-ориентированных_систем', 'расширенная_версия_процесса', 'создание_система', 'постоянное_улучшение']
    Categories: ['software_engineering', 'agile_methodologies', 'systems_design', 'project_management', 'continuous_improvement']

Output:
"topic_7": "Инновационные процессы и развит

2024-12-20 13:54:26,439 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


 "topic_4": "Обработка данных в базах и приложениях"


2024-12-20 13:54:27,187 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


 "topic_90": "Важная реализация кода в практиках Agile и BDD"


Форматированный вывод + комментарии

#### **gpt-4o**
- `"topic_14": "Системные вызовы и ядро Linux"`
- `"topic_5": "Дистрибутивы Linux и сложные задачи"`
- `"topic_4": "Базы данных и обработка данных"`
- `"topic_90": "Рефакторинг и тестирование кода"`

Бейзлайн

#### **llama3**
- `"topic_14": "Системное администрирование и версии ядра"`
- `"topic_5": "Пакеты и дистрибутивы Linux"`
- `"topic_4": "Обработка и анализ данных в базе"`
- `"topic_90": "Реализация кода и официальный сайт"`

Названия 3 и 4 кривые -- не подходит

#### **mistral**
- `"topic_14": "Сборка ядра и версионирование в DevOps"`
- `"topic_5": "Пакеты и рамки параллельного программирования в Linux"`
- `"topic_4": "Обработка данных и базы в front-end разработке"`
- `"topic_90": "Ключевые практики реализации кода"`

Названия 2-4 кривые -- не подходит

#### **aya-expanse**
- `"topic_14": "Управление версиями и системный анализ с акцентом на DevOps"`
- `"topic_5": "Параллельное программирование и распределённые системы в Linux-дистрибутивах"`
- `"topic_4": "Управление данными и базы данных в веб-разработке"`
- `"topic_90": "Важность полного кода и рефакторинга в Agile-практиках"`

Очень развёрнуто, класс

#### **gemma2**
- `"topic_14": "Разработка и тестирование ядра систем"`
- `"topic_5": "Дистрибутивы Linux и параллельное программирование"`
- `"topic_4": "Обработка данных и базы данных"`
- `"topic_90": "Реализация кода и BDD практики"`

Норм

#### **gemma2:27b**
- `"topic_14": "Версии ядра и системное администрирование"`
- `"topic_5": "Дистрибутивы Linux и Параллельное Программирование"`
- `"topic_4": "Обработка данных и базы данных"`
- `"topic_90": "Реализация кода и практики BDD"`

Норм

#### **llama2**
- `"topic_14": "Системное администрирование и версия control"`
- `"topic_5": "Использование технологий в параллельном программировании"`
- `"topic_4": "Обработка данных в базе и приложениях"`
- `"topic_90": "Официальный сайт и реальная реализация кода"`

Все названия, кроме 2-го, звучат криво -- не подходит

#### **mistral:7b-instruct-v0.2-q4_K_S**
- `"topic_14": "Сборка ядерных версий: системное администрирование"`
- `"topic_5": "Пакеты и рамы для дистрибутивов Linux"`
- `"topic_4": "Обработка данных в базах и приложениях"`
- `"topic_90": "Важная реализация кода в практиках Agile и BDD"`

Вывела какую-то хрень в промежутке между ответами, названия звучат криво -- не подходит

По итогу, остались 3 модели -- `aya-expanse`, `gemma2`, `gemma2:27b`

Рекомендую использовать `gemma2`, так как она легковесная + её названия лаконичны по сравнению с `gemma2:27b`

### Итоговый пайплайн

In [ ]:
OLLAMA_CONFIG = {
    "keep_alive": "2m",
    "stream": False,
}

SYSTEM_PROMPT = (
    "You are an AI assistant tasked with generating precise and concise topic titles. "
    "Do not include explanations or reasoning—only provide the final title for each topic. "
    "Write answer in Russian language, but if there are foreign terms leave them as is."
)

SUMMARIZING_PROMPT = """
    You are an AI language model designed to analyze topics defined by keywords, bigrams, and categories, and then summarize each topic into a concise, descriptive name. Use the provided information (keywords, bigrams, and categories) to infer the central theme of each topic and generate a short, meaningful title. Follow these guidelines:

    1. Focus on the most relevant and representative keywords and bigrams to identify the primary subject of the topic.
    2. Use the categories to refine and contextualize the theme.
    3. Avoid overly generic titles; aim for specificity and clarity.
    4. Ensure the title reflects the core idea that unites the keywords, bigrams, and categories.

    Here is the input format:
    ```
    Topic: [topic_name]
    Words: [list of words]
    Bigrams: [list of bigrams]
    Categories: [list of categories]
    ```

    And here is an example of the desired output:
    ```
    Input:
    Topic: topic_15
    Words: ['архитектура', 'код', 'платформа', 'сервис', 'технология']
    Bigrams: ['хранилище_данные', 'облачный_сервис', 'распределённый_система', 'модуль_ядро', 'обработка_больший']
    Categories: ['программирование', 'программная_архитектура', 'распределенные_системы', 'виртуализация', 'paas']

    Output:
    "topic_15": "Архитектура программ и облачные сервисы"

    Input:
    Topic: topic_16
    Words: ['модель', 'текст', 'анализ', 'метод', 'алгоритм']
    Bigrams: ['ред_база', 'учебный_материал', 'субд_ред', 'вычислительный_комплекс', 'анализ_текст']
    Categories: ['алгоритмы', 'наука', 'machine_learning', 'natural_language_processing', 'data_analysis']

    Output:
    "topic_16": "Модели и алгоритмы анализа текста"

    Input:
    Topic: topic_17
    Words: ['проблема', 'требование', 'оценка', 'качество', 'заказчик']
    Bigrams: ['управление_требование', 'больший_количество', 'совместный_работа', 'оценка_качество', 'реализация_проект']
    Categories: ['бизнес_анализ', 'процесс_тестирования', 'управление_требованиями', 'дизайн', 'team_communication']

    Output:
    "topic_17": "Управление требованиями и дизайн"

    Input:
    Topic: topic_18
    Words: ['являться', 'любой', 'образ', 'позволять', 'часть']
    Bigrams: ['возможность_использование', 'высокий_уровень', 'данный_подход', 'код_проект', 'адресный_пространство']
    Categories: ['программирование', 'java', 'разработка_операционных_систем', 'моделирование_бизнес_процессов', 'доклад_со_стенограммой']

    Output:
    "topic_18": "Подходы в программировании и моделировании"

    Input:
    Topic: topic_19
    Words: ['компания', 'технический', 'российский', 'развитие', 'россия']
    Bigrams: ['команда_разработчик', 'ит_компания', 'технический_писатель', 'свой_проект', 'технический_долг']
    Categories: ['agile_process', 'государство_и_софт', 'документирование', 'спо_в_россии', 'планирование']

    Output:
    "topic_19": "Технические проекты в России"
    ```

    Now, process the following topics and generate titles for each:
"""


logging.disable(logging.INFO)

def execute_prompt(prompt, endpoint='http://127.0.0.1:11434/api/generate', model='gemma2', **kwargs):
    text = re.compile(r'#[^\n]*').sub('', prompt).strip()
    response = run_prompt(endpoint, text, model, **kwargs)
    response_content = response.json().get('response', 'No response found.')
    return response_content

In [ ]:
lvl0_titles = []
lvl1_titles = []

for sample in tqdm.tqdm(top_tokens_lvl0_arr):
    outp = execute_prompt(SUMMARIZING_PROMPT + f"```\n{sample}\n```", model="gemma2")
    lvl0_titles.append(outp)

for sample in tqdm.tqdm(top_tokens_lvl1_arr):
    outp = execute_prompt(SUMMARIZING_PROMPT + f"```\n{sample}\n```", model="gemma2")
    lvl1_titles.append(outp)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Postprocessing

In [ ]:
def postprocess_topic_name(topic_name):
    re_pattern = re.compile(r'"topic_\d+":\s*"([^"]+)"')
    return re.match(re_pattern, topic_name).groups()[0]


lvl0_titles_processed = list(map(postprocess_topic_name, lvl0_titles))
lvl1_titles_processed = list(map(postprocess_topic_name, lvl1_titles))

In [ ]:
with open("llm_output/gemma2/levels.json", "w") as fd:
    default_topic_names_lvl0 = [f"topic_{n}" for n in range(len(lvl0_titles_processed))]
    default_topic_names_lvl1 = [f"topic_{n}" for n in range(len(lvl1_titles_processed))]

    json.dump(
        {
            'level0': dict(zip(default_topic_names_lvl0, lvl0_titles_processed)),
            'level1': dict(zip(default_topic_names_lvl1, lvl1_titles_processed)),
        },
        fd
    )

with open("llm_output/gemma2/levels.json", "r") as fd:
    generated_topics = json.load(fd)

generated_topics

{'level0': {'topic_0': 'Agile разработка в IT компаниях',
  'topic_1': 'Возможности использования Linux для программирования',
  'topic_2': 'Использование открытых лицензий в IT',
  'topic_3': 'Автоматизация тестирования и информационной безопасности',
  'topic_4': 'Свободное ПО в образовании и информационных системах',
  'topic_5': 'Дистрибутивы Linux и параллельное программирование',
  'topic_6': 'Специалисты и дискуссии в сфере IT',
  'topic_7': 'Управление проектами и продуктами в компании',
  'topic_8': 'Проектирование мобильных интерфейсов',
  'topic_9': 'Управление бизнес-процессами и возможности пользователей',
  'topic_10': 'Управление серверами и базами данных',
  'topic_11': 'Проектирование устройств и контроллеров',
  'topic_12': 'Разработка ПО и Big Data: Agile подходы',
  'topic_13': 'Курсы программирования для студентов',
  'topic_14': 'Разработка и тестирование ядра систем',
  'topic_15': 'Архитектура распределённых систем и облачных сервисов',
  'topic_16': 'Анализ тек

## Few-shot classification with gemma2

In [ ]:
GEMMA_CLASSIFICATION_PROMPT_PREFIX = f"""<system_prompt>
YOU ARE AN EXPERT IN MULTICLASS MULTILABEL HIERARCHICAL TEXT CLASSIFICATION FOR IT-RELATED CONTENT. YOUR TASK IS TO ANALYZE TEXTS FROM IT CONFERENCES (IN RUSSIAN AND ENGLISH) OF VARYING LENGTH (FROM SHORT TITLES TO DETAILED ARTICLES) AND ASSIGN THE MOST RELEVANT HIERARCHICAL CATEGORIES BASED ON PROVIDED LABELS AND THEIR RELATIONSHIPS.

### INSTRUCTIONS ###

1. **READ AND UNDERSTAND THE TEXT**: 
   - IDENTIFY the key topics, themes, and context of the text.
   - DISTINGUISH between the main focus and supplementary details.

2. **USE THE PROVIDED HIERARCHICAL CATEGORY STRUCTURE**:
   - ALWAYS ASSIGN CATEGORIES from the provided list.
   - IF THE TEXT IS SHORT, prioritize HIGH-LEVEL and GENERAL categories.
   - IF THE TEXT IS LONG, assign DETAILED, LOW-LEVEL, and SPECIFIC categories.

3. **MAINTAIN CONSISTENCY**:
   - SELECT categories based on the text content and not assumptions.
   - ENSURE the categories are logically consistent with the hierarchy.

4. **OUTPUT FORMAT**:
   - RETURN ONLY a Python-style list of categories (e.g., `['Category1', 'Category2']`).
   - DO NOT include explanations, reasoning, or logs in the response.

5. **CONSIDER CONTEXTUAL RELEVANCE**:
   - ACCOUNT for both linguistic (Russian/English) and domain-specific terminology.
   - INCLUDE categories related to both technical and thematic aspects of the text.

6. **PRIORITIZATION RULES**:
   - PRIORITIZE categories that capture the technical or thematic core of the text.
   - AVOID unrelated or overly broad categories unless the text explicitly requires them.
   - DO NOT ASSIGN categories without clear support in the text.
   - DO NOT INCLUDE reasoning, analysis, or commentary in the output.

### CHAIN OF THOUGHTS ###

1. **UNDERSTAND THE TEXT**:
   - READ the text carefully to grasp the main idea.
   - IDENTIFY specific keywords and phrases pointing to IT-related topics.

2. **DETERMINE CATEGORY SCOPE**:
   - CLASSIFY short texts with a focus on overarching themes or event types.
   - CLASSIFY long texts with a focus on detailed, specific technical topics.

3. **FILTER CATEGORIES**:
   - USE a subset of categories based on the keywords and context in the text.
   - PRIORITIZE specific and detailed categories for technical texts.

4. **ASSIGN RELEVANT CATEGORIES**:
   - CHOOSE categories with direct support in the text.
   - IF THE TEXT IS AMBIGUOUS OR GENERAL, assign higher-level categories.

5. **VALIDATE SELECTIONS**:
   - CHECK for logical consistency.
   - REMOVE any categories that are redundant or insufficiently supported.

6. **FORMAT THE OUTPUT**:
   - STRUCTURE the list clearly in Python list format.
   - INCLUDE all directly relevant categories.

7. **REVIEW AND FINALIZE**:
   - ENSURE all chosen categories align with the hierarchy and text content.
   - VERIFY the list is complete and free of errors.

### CATEGORIES ###
{list(ALL_CATEGORIES_WITHOUT_NAMES)}

### EXAMPLES ###

#### Input Sample:
3 практики креативного мышления в разработке пользовательских интерфейсов (Владимир Завертайлов, SECR-2013) <sep> В докладе рассмотрены креативные методики «Визуальный бриф», «Метод Уолта Диснея» и «SCAMPER», и применение этих методик для разработки интерфейсов и дизайна программных продуктов (web-систем и мобильных приложений).

#### Output:
['UX проектирование', 'Прототипирование UI']

---

#### Input Sample:
Design of PyTest-based Framework for Modules that Use RabbitMQ and REST (Никита Ковригин, SECR-2017) <sep> How to achieve fast test development process and make it easy for QA engineers to write tests and don’t think about difficulties? How to save our time and resources if test objects are “big” and “complex” using pytest? We have our own experience and want to show you how we solved it.

#### Output:
['Talks in English', 'Автоматизированное тестирование']

---

#### Input Sample:
Применение популярных протоколов и свободного ПО в управлении мобильным роботом (Андрей Дунец, LVEE-2014) <sep> We consider the problems of telemetry, positioning and control of the mobile robot for monitoring of rivers and lakes. The project used Bluetooth technology for the transmission of telemetry data. RTKLIB library implementation of Real-Time Kinematic algorithm used for positioning. The current version of the control system is based on the Ardupilot project.

#### Output:
['Arduino', 'Робототехника']

---

### WHAT NOT TO DO ###

- **DO NOT** ASSIGN CATEGORIES OUTSIDE THE PROVIDED LIST.
- **DO NOT** ASSIGN CATEGORIES WITHOUT CLEAR TEXTUAL SUPPORT.
- **DO NOT** DUPLICATE HIGH-LEVEL AND LOW-LEVEL CATEGORIES IN THE SAME LIST (E.G., 'AGILE' AND 'AGILE-КУЛЬТУРА').
- **DO NOT** FAIL TO ADAPT CATEGORY SELECTION BASED ON TEXT LENGTH AND DETAIL.

### FINAL OUTPUT FORMAT ###

- Python-style list of categories (e.g., `['Category1', 'Category2']`).
- INCLUDE only categories directly relevant to the content.

### USER INPUT ###
- TEXT: 
"""

GEMMA_CLASSIFICATION_PROMPT_SUFFIX = """
### OUTPUT ###
- Python-style list of categories only.

</system_prompt>
"""

In [ ]:
np.random.seed(123)
rand_ids = np.random.choice(len(processed_data), 4, replace=False)
rand_text_samples = [elem['text'] for elem in np.array(processed_data)[rand_ids]]
rand_targets = [elem['categories'] for elem in np.array(processed_data)[rand_ids]]

In [ ]:
for rand_text, rand_trg in zip(rand_text_samples, rand_targets):
    print(rand_text[:100])
    print('Generated target')
    debug_prompt(GEMMA_CLASSIFICATION_PROMPT_PREFIX + "\"" + rand_text[:5000] + "\"" + GEMMA_CLASSIFICATION_PROMPT_SUFFIX, model='gemma2')
    print('True target')
    print(rand_trg)
    print('='*80)

Альтернатива горбатому RPM — двугорбый RPM (Глеб Фотенгауэр-Малиновский, OSSDEVCONF-2016) <sep> Испо
Generated target


2024-12-24 20:12:24,857 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


['Linux', 'Software Development'] 

True target
['Linux-дистрибутивы']
Mine Your Own Business — Using Process Mining to Turn Big Data into Real Value (Wil van der Aalst, S
Generated target


2024-12-24 20:12:26,943 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


```python
['Process Mining', 'Business Process Management (BPM)', 'Workflow Management (WfM)', 'Data Mining (DM)', 'Business Intelligence (BI)', 'Machine Learning (ML)', 'Big Data'] 
```
True target
['Моделирование бизнес-процессов']
Дизайн в разработке (Роман Квартальнов, SECR-2015) <sep> Из доклада вы узнаете, как можно сократить 
Generated target


2024-12-24 20:12:28,055 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


['Дизайн', 'UX проектирование'] 

True target
['Процесс разработки UX и UI']
Использование системы выделенных признаков для задач поиска по исходному тесту (Алексей Пустыгин, OS
Generated target


2024-12-24 20:12:29,205 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


```python
['method', 'class']
```
True target
['Open-source', 'Образование']
